In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import os
import pandas as pd
from datetime import datetime
import geopandas as gpd
from pyproj import Proj, transform
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from geopandas.tools import sjoin

#from src.hydra import mongo_utils
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen 
from src.sentiment.senti_utils import *
from src.scripts.process_tweets import *
from labMTsimple.storyLab import *

In [5]:
gdf = gpd.read_file('../data/raw/geo/ParkServe_shp_DataShare_05152019/ParkServe_shp_DataShare_05152019/ParkServe_Parks_05152019.shp')
park_codes =pd.read_csv("../data/processed/tpl/park_designations.csv", index_col="code") 
park_codes['value'] = park_codes['value'].str.replace(" ","_").str.lower()

In [48]:
cities = Path("../data/processed/park_user_tweets/").glob("*.json")
city_ids = []
for city in cities:
    city_ids.append(city.stem[-7:])
#tweets = load_tweets(city)
#print("Total tweets in {}: {}".format(city.stem,len(tweets)))

In [51]:
parks = gdf[gdf.Park_Pla_1.isin(city_ids)].copy()
parks_df = pd.DataFrame(parks.drop(columns='geometry', axis=1))

In [67]:
park_info = parks[['ParkID','Park_Place','Park_Pla_1','Park_Desig','Park_Size_']].copy()
city_names = park_info[['Park_Pla_1','Park_Place']].copy().drop_duplicates().set_index('Park_Pla_1')
park_counts = park_info[['Park_Pla_1','Park_Desig']].pivot_table(index='Park_Pla_1', columns='Park_Desig', aggfunc=len, fill_value=0)
park_totals = park_counts.sum()
park_info.head()

,ParkID,Park_Place,Park_Pla_1,Park_Desig,Park_Size_
1207,24040000001,Baltimore city,2404000,LP,0.168729
1208,24040000002,Baltimore city,2404000,LP,0.622871
1209,24040000003,Baltimore city,2404000,LP,1.357271
1210,24040000004,Baltimore city,2404000,LP,0.992805
1211,24040000005,Baltimore city,2404000,LP,1.725284


In [71]:
common_parks = park_totals[park_totals>100].index.tolist()
common_counts =park_counts[common_parks].copy()
common_counts.rename(columns=park_codes['value'].to_dict(),inplace=True)
city_park_counts = city_names.join(common_counts)
city_park_counts.head()

,Park_Place,local_conservation_area,local_other_or_unknown,local_park,local_recreation_area,national_public_lands,state_park
Park_Pla_1,,,,,,,
2404000,Baltimore city,0,0,286,7,0,0
3712000,Charlotte city,7,0,189,18,0,0
3916000,Cleveland city,0,3,153,25,0,3
3918000,Columbus city,70,3,350,8,0,0
2622000,Detroit city,0,1,310,43,0,0


In [100]:
park_info = parks[['ParkID','Park_Place','Park_Pla_1','Park_Desig','Park_Size_']].copy()
size = park_info[['ParkID','Park_Size_']].copy()
size['acres_lt_1'] =((size['Park_Size_']>0) & (size['Park_Size_']<=1))
size['acres_1_10'] =((size['Park_Size_']>1) & (size['Park_Size_']<=10))
size['acres_10_100'] =((size['Park_Size_']>10) & (size['Park_Size_']<=100))
size['acres_gt_100'] =(size['Park_Size_']>100)
size = pd.melt(size, id_vars=['ParkID'], value_vars = size.columns[2:],var_name="size_cat")
size = size[size.value]
size.set_index('ParkID',inplace=True)

,size_cat,value
ParkID,,
24040000001,acres_lt_1,True
24040000002,acres_lt_1,True
24040000004,acres_lt_1,True
24040000007,acres_lt_1,True
24040000011,acres_lt_1,True


In [101]:
park_info_size = park_info.set_index('ParkID').join(size.size_cat)
park_info_size.head()

,Park_Place,Park_Pla_1,Park_Desig,Park_Size_,size_cat
ParkID,,,,,
24040000001,Baltimore city,2404000,LP,0.168729,acres_lt_1
24040000002,Baltimore city,2404000,LP,0.622871,acres_lt_1
24040000003,Baltimore city,2404000,LP,1.357271,acres_1_10
24040000004,Baltimore city,2404000,LP,0.992805,acres_lt_1
24040000005,Baltimore city,2404000,LP,1.725284,acres_1_10


In [108]:
park_info_size['park_type'] = park_info_size['Park_Desig'].map(park_codes.to_dict()['value'])
park_info_size.head()

,Park_Place,Park_Pla_1,Park_Desig,Park_Size_,size_cat,park_type
ParkID,,,,,,
24040000001,Baltimore city,2404000,LP,0.168729,acres_lt_1,local_park
24040000002,Baltimore city,2404000,LP,0.622871,acres_lt_1,local_park
24040000003,Baltimore city,2404000,LP,1.357271,acres_1_10,local_park
24040000004,Baltimore city,2404000,LP,0.992805,acres_lt_1,local_park
24040000005,Baltimore city,2404000,LP,1.725284,acres_1_10,local_park


In [ ]:
park

In [109]:
park_info_size.to_csv('results/park_info_size.csv')